# Generative AI playground
Quick start: pip install -q -U google-genai


In [ ]:
import config

## Code base: basic text generation

In [5]:
from google import genai


client = genai.Client(api_key=config.apikey)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain how AI works in less than 100 words",
)

print(response.text)

AI mimics human intelligence using computer programs. It learns from vast amounts of data through algorithms like neural networks. These algorithms identify patterns, make predictions, and improve over time without explicit programming. Common AI applications include image recognition, natural language processing, and problem-solving.

